In [ ]:

import pandas as pd
import os
import datetime as dt
from helper import *
from paths import *

enc = 'cp850'
sep = ';'

In [ ]:

hj_nr, prev_start, prev_end = get_half_year_info().values()
last_HJ = f'Z{hj_nr}'
prev_start = pd.to_datetime(prev_start)
prev_end = pd.to_datetime(prev_end)

In [ ]:

kunden_segments = pd.read_excel(ks_path)
kunden_segment_dict = dict(zip(kunden_segments['Alt'],kunden_segments['Neu']))

kw = pd.read_csv(kw_path, sep=sep,encoding=enc,on_bad_lines='skip')
adresse = pd.read_csv(adresse_path, sep=sep,encoding=enc)
stat = pd.read_csv(stat_path, sep=sep,encoding=enc,usecols=['NUMMER','ERSTKAUF'])

inx = pd.read_excel(inx_path,usecols=['NUMMER','NL_TYPE'])

rechnung = pd.read_csv(rech_f01,sep=sep,encoding=enc)

In [ ]:
kw[last_HJ] = kw[last_HJ].map(kunden_segment_dict)

In [ ]:

kw['NUMMER'] = process_id(kw['NUMMER'])

adresse['NUMMER'] = process_id(adresse['NUMMER'])
adresse['GEBURT'] = process_date(adresse['GEBURT'])
adresse['SYS_ANLAGE'] = process_date(adresse['SYS_ANLAGE'])
adresse = assign_age(adresse)
adresse = assign_sources(adresse,'QUELLE')
adresse['QUELLE'] = adresse['SOURCE']
adresse = adresse.drop(columns=['SOURCE'])

stat['NUMMER'] = process_id(stat['NUMMER'])
stat['ERSTKAUF'] = process_date(stat['ERSTKAUF'])

inx['NUMMER'] = process_id(inx['NUMMER'])
rechnung['NUMMER'] = process_id(rechnung['NUMMER'])
rechnung['AUF_ANLAGE'] = process_date(rechnung['AUF_ANLAGE'])
rechnung = assign_sources(rechnung,'MEDIACODE')
rechnung['MEDIACODE'] = rechnung['SOURCE']
rechnung = rechnung.drop(columns=['SOURCE'])
rechnung['HERKUNFT'] = rechnung['HERKUNFT'].astype(str).str.replace('.0','')

rechnung['HERKUNFT'] = rechnung['HERKUNFT'].map(herkunft)
rechnung['HERKUNFT'].astype(str).value_counts()

In [ ]:
adresse['ANREDE'] = process_anrede(adresse['ANREDE'])
adresse['ANREDE'] = adresse['ANREDE'].map(anrede)

In [ ]:
rechnung = rechnung[(rechnung['AUF_ANLAGE']>= prev_start)&(rechnung['AUF_ANLAGE']<= prev_end)]
rechnung_gr = rechnung.groupby(['NUMMER']).agg(             ANZ_AUF=('AUFTRAG_NR','nunique'),
                                                            AUF_ANLAGE=('AUF_ANLAGE','first'),
                                                            MEDIACODE=('MEDIACODE','first'),
                                                            UMSATZ=('PREIS','sum'),
                                                            HERKUNFT=('HERKUNFT','first')
                                                            ).reset_index()

In [ ]:

df = pd.merge(adresse,kw[['NUMMER',last_HJ]],on='NUMMER',how='left')
df = pd.merge(df,stat,on='NUMMER',how='left')
df = pd.merge(df,inx,on='NUMMER',how='left')
df = pd.merge(df,rechnung_gr,on='NUMMER',how='left')


In [ ]:
df = df[['NUMMER', 'ANREDE', 'TITEL', 'VORNAME', 'NAME', 'QUELLE', 'LKZ', 'PLZ', 'ORT',
       'SYS_ANLAGE', 'AGE_GROUP',  last_HJ, 'ERSTKAUF', 'NL_TYPE',
       'ANZ_AUF', 'HERKUNFT',  'MEDIACODE','AUF_ANLAGE', 'UMSATZ']]
df = df.rename(columns={last_HJ:'Kundengruppe'})
df = df.drop_duplicates(subset='NUMMER')

In [ ]:
# df.loc[(df['ERSTKAUF']>=prev_start)&(df['ERSTKAUF']<prev_end),'Kundengruppe'] = "Neukunden-1"
df.loc[(df['ERSTKAUF'].isna()),'Kundengruppe'] = "Interessenten"


In [ ]:
df = df[df['SYS_ANLAGE']<prev_end]

In [ ]:
kg

In [ ]:
# df.to_csv('kw.csv',sep=';',encoding='cp850',index=False)

In [ ]:
kg = list(df[df['Kundengruppe'].notna()]['Kundengruppe'].unique())
with pd.ExcelWriter('KundenInfo-2024-2HJ.xlsx',engine='xlsxwriter') as writer:
    for item in kg:
        print(item)
        df[df['Kundengruppe'] == item].to_excel(writer,index=False,sheet_name=item)

In [ ]:
# df[df['Kundengruppe'] == 'Neukunden-2'].to_excel('aktiv-top-2024-2hj.xlsx',index=False)

In [ ]:
df[df['Kundengruppe'] == 'Neukunden-2']

In [ ]:
kw[['NUMMER','Z49']]